# クラス・テンプレート


## キーポイント

* クラステンプレートは、「クラスが必要とする１つ以上の型や値を、テンプレート引数で指定できるようにしたもの」
* ひとつのテンプレート引数`T`を持つクラステンプレートは、次のように書く
    ```cpp
    template<typename T>
    class クラス名 {};
    ```
* テンプレート引数は、カンマで区切って必要なだけ追加できる
    ```cpp
    template<typename T1, typename T2, typename T3>
    class クラス名 {};
    ```
* クラスを設計する場合、最初に「必要なインターフェイス」を決める
* クラスの外から勝手に操作されるとまずいメンバ変数は`private`メンバにする
* 標準ライブラリのコンテナは「コンテナ間で名前と機能が共通の型」を定義している
* `explicit`キーワードを使うと暗黙の型変換を無効化できる
* インクルードガードを使うと、同じヘッダファイルを何度インクルードしてもコンパイルエラーにならない


----

## 1 vectorクラスを自作する

----


### 1.1 この章の方針

以前に、クラスとはどんなものか、ということを説明しました。<br>
しかし、「クラスの作り方」については詳しく説明していませんでした。

クラスの設計には、そのクラスのオブジェクトの作成において必要なことだけでなく、コピーされるときや代入されるとき、また破棄されるときにするべきことも含まれます。

メンバ関数を持たない、単純な構造体のようなクラスであれば、これらのことはコンパイラに任せられます。しかし、ある程度の複雑さを持つクラスでは、これらの操作が実行されたときの振る舞いを、プログラマが適切に設計しなくてはなりません。

この章では、標準ライブラリの`vector`型をお手本に、同様のクラスを作成することで、クラスの設計と作成の方法を解説します。<br>
もちろん、`vector`が持つすべての機能を作成するのは大変すぎます。そこで、特に重要な機能に限定して作成することにします。<br>
つまり、「`vector`の機能限定版」を作るわけです。

作成するクラスの名前は、`vector`と混同しないように`EasyVec`(イージー・ベク、「簡易な配列」という意味)とします。<br>
比較的簡単なメンバ関数から作り始めて、最終的にコピー、代入、破棄までを作っていきます。

>**【本テキストの心構え】**<br>
>このテキストの大半は、プログラムを書き写すだけで完了します。<br>
>ですが、ただ書き写すだけでは、クラスの作り方は身につきません。<br>
>テキストを読んで書かれていることを理解し、このようなプログラムになっている理由を考えるよう努めてください。


### 1.2 クラスのインターフェイスを考える

クラスを設計する場合、最初に「必要なインターフェイス」を決めます。

インターフェイスとは、「そのクラスを使ってできる動作や作用」のことです。<br>
インターフェイスには、パブリックなメンバ関数やメンバ変数の宣言が含まれます。

ですが、いきなり「最初にインターフェイスを決めてください」と言われても、どこから手を付けていいのか分からないと思います。<br>
ひとつの方法は、「そのクラスを使って書けるプログラムを想像してみる」ことです。

今回は、お手本として`vector`型があります。そこで、これまで`vector`をどのように使ってきたかを見直してみましょう。

```cpp
// vector型の変数宣言
vector<string> a;
vector<int> b(10);

// サイズと添え字を使ったループ
for (int i = 0; i < (int)a.size(); i++) {
  cin >> a[i];
}

// イテレータを使う
sort(a.begin(), a.end());

// 要素の追加と削除
a.push_back("first");
a.pop_back();
```

上記のプログラムでは、`vector`の以下の機能を使用しています。

1. パラメータなしの変数宣言
2. サイズを指定して変数宣言
3. サイズの取得
4. 添字によるデータの読み書き
5. イテレータの取得
6. 要素の追加と削除

`EasyVec`クラスがこれらの機能を備えていれば、`vector`を`EasyVec`に置き換えても実行できるはずです。<br>
つまり、この6つが`EasyVec`クラスの「インターフェイス」ということになります。

もちろん、`vector`型には上記で挙げた以外にもさまざまな機能があります。<br>
ですが、すべての機能を作成するにはかなりの時間が必要なので、ここで作成するのは上記の機能に限定します。


#### ファイルの準備

さて、本テキストでは、実際に`EasyVec`クラスを作成してもらいます。<br>
クラスを作成するときは、「クラス用のヘッダファイルとソースファイル」を用意します。<br>
ファイル名は「クラス名.h」と「クラス名.cpp」とすることが多いです。

ただし、テンプレートを使う場合は全部ヘッダファイルに書きます。<br>
`vector`クラスもテンプレートを使っているので、`EasyVec`もテンプレートを使うことになります。<br>
名前は`EasyVec`なので、ヘッダファイル名は`EasyVec.h`とします。

`Google Colab`では、コードセルの先頭に`%%writefile ファイル名`というマジックコマンドを書くことでファイルを作成できます。

この下に、マジックコマンドを書いた`EasyVec.h`というコードセルをご用意しました。

タイトルの`EasyVec.h`という文字をクリックすると、右側にアイコンリストが表示されます。アイコンリストから「タブのミラーセル」という説明が出るアイコンをクリックして、セルを複製してください(他のアイコンをクリックしないように注意)。


In [ ]:
# @title EasyVec.h
# タブの「ミラーセル」機能で、このセルを複製してください。
%%writefile EasyVec.h


ウィンドウの右側に、`EasyVec.h`というタブが表示されていたら準備完了です。以後、「`EasyVec.h`に次のプログラムを追加(あるいは削除、変更)してください」というテキストを見たら、右に表示されている`EasyVec.h`タブのコードを変更してください。<br>
変更後に`▶`ボタンをクリックして、ファイルを保存すること。


### 1.3 EasyVecクラスの定義

インターフェイスが決まったので、次は`EasyVec`クラスを具体的に定義していきます。<br>
`vector`のように、さまざまな型の配列を作れるようにするには、`EasyVec`クラスを「クラス・テンプレート」として定義します。

クラステンプレートは、「クラスが必要とする１つ以上の型や値を、テンプレート引数で指定できるようにしたもの」です。<br>
例えば、標準ライブラリの`vector`や`unordered_map`、`list`などは、すべてクラステンプレートです。

クラステンプレートの定義には、関数テンプレートと同じく`template`キーワードと`typename`キーワードを使います。

`EasyVec.h`の`%%writefile`行の下に、次のプログラムを追加してください。追加したら`▶`ボタンを押して保存すること。

```cpp
/*
* 配列クラス
*/
template<typename T>
class EasyVec {
};
```

このプログラムは、「`T`というテンプレート引数を持つ、`EasyVec`という名前のクラステンプレート」を定義しています。

>追加のしかたが分からない場合は、以下の「追加例」の「コードの表示」をクリックして、参考にしてください


In [ ]:
# @title 🧩追加例
%%writefile EasyVec.h
/*
* 配列クラス
*/
template<typename T>
class EasyVec {
};

#### EasyVecが管理するデータ

`vector`型は配列を扱うクラスです。`EasyVec`クラスは`vector`の機能限定版なので、配列を扱える必要があります。<br>
それには、配列を扱うためのメンバ変数を持っていなくてはなりません。

また、`vector`型と同様の`begin`, `end`, `size`という3つのメンバ関数を作る予定なので、メンバ変数はこれらにも対応できなくてはなりません。そのためには、「配列の先頭」、「配列の終端」、「配列のサイズ」のうち、少なくとも2つのデータが必要です。

どの2つを選んでもいいのですが、今回は「配列の先頭」と「配列の終端」のペアを使うことにします。<br>
「配列の先頭」をあらわす変数名は`first`(ファースト)、「配列の終端」をあらわす変数名は`last`(ラスト)とします。

`first`と`last`、そして配列の関係は次の図のようになります。

<img width="450px" hspace="50px" src="https://raw.githubusercontent.com/tn-mai/cpp2025/refs/heads/main/images/EasyVec_data_and_end.drawio.png" />

これらのメンバ変数は「プライベート」メンバとします。<br>
メンバ変数をクラスの外から勝手に操作されると、データの整合性が取れなくなるからです。<br>
例えば、`end`の値を勝手に変えられると、配列の終端がわからなくなり、サイズの計算もできなくなります。

それでは、`EasyVec.h`に次のプログラムを追加してください。

>以下のプログラムはdiff(ディフ)という表記法を使っています。diffでは、先頭に`+`(プラス)記号が付いていると「元のプログラムに追加する行」という意味になります。また、`-`(マイナス)記号が付いていると「元のプログラムから削除する行」という意味になります。<br>
>diffを書き写すときは、先頭の+記号は書かないでください。例えば、
>
><pre><code><div><font color="#3a3">+private:</font>
></div></code></pre>
>
>を書き写すときは、先頭の+を抜いた残りの
>
><pre><code><div>private:</div></code></pre>
>
>の部分だけを書き写します。

**diff**

<pre><code><div> /*
 * 配列クラス
 */
 template&lt;typename T&gt;
 class EasyVec {
<font color="#3a3">+private:
+  T* first = nullptr; // 配列の先頭
+  T* last = nullptr;  // 配列の終端</font>
 };
</div></code></pre>

`first`と`last`には、初期値として`nullptr`を指定しました。<br>
最初はメモリを確保していないので、これらの変数には指すべきメモリがないからです。<br>
`nullptr`を指定することで、これらのメンバ変数が「何も指していない」ことを表現できます。


In [ ]:
# @title 🧩追加例
%%writefile EasyVec.h
/*
* 配列クラス
*/
template<typename T>
class EasyVec {
private:
  T* first = nullptr; // 配列の先頭
  T* last = nullptr;  // 配列の終端
};

#### メモリの確保

`EasyVec`クラスは、配列用のメモリを動的に確保します。<br>
本来は`shared_ptr`を使うべき場面ですが、今回はできるだけ`vector`型と同じ設計にしたいです。<br>
そこで、メモリの確保もあえて`shared_ptr`を使わず、`vector`型と同じ方法を選ぶことにします。

さて、`vector`型は、メモリの確保に標準ライブラリの`allocator`(アロケータ)クラスを使っています。<br>
`allocator`クラスは次のように宣言されています。

```cpp
template<typename T>
class allocator;
```

`allocator`クラスの機能を使うには、`allocator`クラスの変数を宣言します。<br>
また、`allocator`クラスを使うには、`memory`ヘッダをインクルードする必要があります。

`EasyVec.h`に次のプログラムを追加してください。

**diff**

<pre><code><div><font color="#3a3">+#include &lt;memory&gt;
+</font>
 /*
 * 配列クラス
 */
 template&lt;typename T&gt;
 class EasyVec {
 private:
   T* first = nullptr; // 配列の先頭
   T* last = nullptr;  // 配列の終端
<font color="#3a3">+  std::allocator&lt;T&gt; alloc; // メモリ管理用のオブジェクト</font>
 };
</div></code></pre>






In [ ]:
# @title 🧩追加例
%%writefile EasyVec.h
#include <memory>

/*
* 配列クラス
*/
template<typename T>
class EasyVec {
private:
  T* first = nullptr; // 配列の先頭
  T* last = nullptr;  // 配列の終端
  std::allocator<T> alloc; // メモリ管理用のオブジェクト
};

### 1.4 型の定義

`vector`型に限らず、標準ライブラリのコンテナは、「コンテナ間で名前と機能が共通の型」をいくつか定義しています。<br>
それらの型の主な目的は、「コンテナが違っても共通のコードを書きやすくするため」です。

実際に、汎用アルゴリズムはこれらの型を活用して書かれています。<br>
`EasyVec`でも汎用アルゴリズムを活用したければ、それらの型を定義する必要があります。必要なのは次の4つです。

* iterator:&emsp;&emsp;&emsp;基本となるイテレータ型
* const_iterator: 指している値を書き換えられないイテレータ型
* size_type:&emsp;&emsp; 配列の大きさなどを表現する型
* value_type:&emsp;&ensp;コンテナに格納するデータ型

これらの型の定義で難しいところは、「どの型を選ぶか」という点です。<br>
例えば、`EasyVec`の場合、`vector`型と同様に、配列としての機能を持たせたいわけです。<br>
そうなると、`iterator`の型は、`vector`と同じ「隣接イテレータ」でなくてはならないでしょう。

`EasyVec`用の隣接イテレータクラスを作ることもできますが、配列の場合はポインタで代用できます。<br>
そこで、イテレータの型は`T*`とするのが適切でしょう。

と、このように、ひとつの型を決めるだけでも、いろいろなことを考える必要があります。

`const_iterator`の型は、`iterator`に「指している値を書き換えられない」という属性が付いたものです。<br>
これは、`T*`に`const`キーワードを付けて`const T*`とすることで実現できます。

`size_type`は特別な理由がない限り、C++で標準的な大きさをあらわす型である`size_t`型にします。

`value_type`は配列のデータをあらわすので、`T`型です。

それでは、これらの型を追加しましょう。`EasyVec.h`に、次のプログラムを追加してください。

**diff**

<pre><code><div> #include &lt;memory&gt;

 /*
 * 配列クラス
 */
 template&lt;typename T&gt;
 class EasyVec {<font color="#3a3">
+public:
+  // 型の定義
+  using iterator = T*;
+  using const_iterator = const T*;
+  using size_type = size_t;
+  using value_type = T;
+</font>
 private:
   T* first = nullptr; // 配列の先頭
   T* last = nullptr;  // 配列の終端
   std::allocator&lt;T&gt; alloc; // メモリ管理用のオブジェクト
 };
</div></code></pre>

このプログラムでは、型の定義に`using`(ユージング)エイリアスという機能を使っています。<br>
エイリアスは「別名」という意味で、その名のとおり「型に別名を付ける」機能です。<br>
`using`エイリアスは次のように書きます。

`using 別名 = 元になる型名;`

`=`記号の左側が「別名」で、右側が「元になる型(クラス)の名前」です。<br>
例えば、先ほどのプログラムの`using iterator = T*`は、

&emsp;**`T*`(Tのポインタ)型の別名として、`iterator`(イテレータ)という名前を定義**

しています。以後は、`iterator`と書いたら、それは`T*`と同じものとして扱われます。


In [ ]:
# @title 🧩追加例
%%writefile EasyVec.h
#include <memory>

/*
* 配列クラス
*/
template<typename T>
class EasyVec {
public:
  // 型の定義
  using iterator = T*;
  using const_iterator = const T*;
  using size_type = size_t;
  using value_type = T;

private:
  T* first = nullptr; // 配列の先頭
  T* last = nullptr;  // 配列の終端
  std::allocator<T> alloc; // メモリ管理用のオブジェクト
};

### 1.5 コンストラクタ

次に、2つのコンストラクタを定義します。<br>
ひとつは引数を持たないデフォルトコンストラクタ、もうひとつは配列の大きさを指定するコンストラクタです。

とりあえず、何もしないコンストラクタを作成しましょう。`EasyVec.h`に、次のプログラムを追加してください。

**diff**

<pre><code><div> #include &lt;memory&gt;

 template&lt;typename T&gt;
 class EasyVec {
 public:
   // 型の定義
   using iterator = T*;
   using const_iterator = const T*;
   using size_type = size_t;
   using value_type = T;
<font color="#3a3">+
+  // コンストラクタ
+  EasyVec() = default;
+  EasyVec(size_t n) {
+  }</font>

 private:
   T* first = nullptr; // 配列の先頭
   T* last = nullptr;  // 配列の終端
   std::allocator<T> alloc; // メモリ管理用のオブジェクト
 };
</div></code></pre>

コンストラクタを作るときに重要なことは、「全てのメンバ変数を適切に初期化する」ことです。<br>
最初の状態が間違っていると、メンバ関数が正常に動作できないからです。

`EasyVec`クラスの場合、「必要に応じて配列用のメモリを確保し、すべてのメンバ変数に適切な初期値を指定すること」となります。

`allocator`クラスは自身のコンストラクタによって適切に設定されるため、何もする必要はありません。<br>
そのため、`alloc`メンバ変数には何もしません。設定が必要なのは`first`と`last`の2つだけです。

デフォルトコンストラクタには何も書く必要はありません。`first`と`last`には適切な初期値が指定されているからです。<br>
そのため、`default`指定しています。

引数付きのコンストラクタは、`n`個の要素を管理するメモリを動的に確保し、次に各要素を初期化しなくてはなりません。<br>
`allocator`クラスを使ってメモリを確保するには、`allocate`(アロケート)メンバ関数を使います。

>**書式**
>
>```cpp
>T* allocator::allocate(size_t n);
>```
>
>**引数**
>
>* n&emsp;確保する要素数
>
>**戻り値**
>
>`T`型の`n`個分の連続したメモリ領域を確保し、そのアドレスを返します。

また、`allocator`を使ったメモリ確保は少し特殊で、`T`型のコンストラクタが呼び出されません。<br>
確保したメモリ領域に対してコンストラクタを呼び出すには、`construct_at`(コンストラクト・アット)関数を使います。

>**書式**
>
>```cpp
>template<typename T, typename... Args>
>T* construct_at(T* p, Args&... args);
>```
>
>**引数**
>
>* p&emsp;&ensp;初期化するオブジェクトのアドレス
>* args コンストラクタに渡す引数リスト(省略可能)
>
>**戻り値**
>
>`p`を返します。
>
>**注意**
>
>`construct_at`関数はC++20で追加されました。<br>
>Visual Studio等で使うには、使用言語をC++20以上に設定する必要があります。

それでは、`EasyVec.h`に次のプログラムを追加してください。

**diff**

<pre><code><div> template&lt;typename T&gt;
 class EasyVec {
 public:
   // 型の定義
   using iterator = T*;
   using const_iterator = const T*;
   using size_type = size_t;
   using value_type = T;

   // コンストラクタ
   EasyVec() = default;
   EasyVec(size_t n) {<font color="#3a3">
+    // メモリを確保し、配列の先頭を設定
+    first = alloc.allocate(n);
+
+    // 要素を初期化
+    for (size_t i = 0; i < n; i++) {
+      std::construct_at(first + i);
+    }
+
+    // 配列の終端を設定
+    last = first + n;</font>
   }

 private:
   T* first = nullptr; // 配列の先頭
   T* last = nullptr;  // 配列の終端
   std::allocator&lt;T&gt; alloc; // メモリ管理用のオブジェクト
 };
</div></code></pre>

`EasyVec.h`にプログラムを追加し終わったら、以下の動作テストを実行してください。<br>
プログラムに問題がなければ`AC`が出力されます。


In [ ]:
# @title 動作テスト
!echo -e "#include \"EasyVec.h\"\nint main() {\nEasyVec<int> a;\nEasyVec<double> b(5);\n}\n">test.cpp
!g++ -otest -std=c++20 -Wall -Wextra test.cpp && ./test > /dev/null && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

### 1.6 デストラクタ

メモリを確保した場合、どこかのタイミングで、確保したメモリを解放しなくてはなりません。<br>
解放しないでいると「メモリ・リーク」になるからです。

クラス内で確保したメモリの場合、デストラクタで解放すると確実です。<br>
デストラクタは「変数が削除されるときに必ず呼び出される」からです。<br>
デストラクタにちゃんとした解放プログラムを書けば、メモリ・リークを完全に防げます。

さて、`allocator`クラスを使ってメモリを解放するには、`deallocate`(デアロケート)メンバ関数を使います。

>**書式**
>
>```cpp
>void allocator::deallocate(T* p, size_t n);
>```
>
>**引数**
>
>* p&emsp;解放するメモリアドレス
>* n&emsp;`allocate`で確保したときの要素数
>
>**戻り値**
>
>なし。

`allocate`メンバ関数がコンストラクタを呼び出さないように、`deallocate`メンバ関数もデストラクタを呼び出しません。<br>
デストラクタを呼び出し、オブジェクトを破棄するには`destroy_at`(デストロイ・アット)関数を使います。

>**書式**
>
>```cpp
>template<typename T>
>void destroy_at(T* p);
>```
>
>**引数**
>
>* p&emsp;破棄するオブジェクトのアドレス
>
>**戻り値**
>
>なし。

上記の2つの関数を使って、デストラクタを作成しましょう。`EasyVec.h`に次のプログラムを追加してください。

なお、全部のコードを書くと長すぎるので、今後の`diff`では追加するコードの前後の部分だけを記載します。<br>
追加するコードの位置は、前後のコードから推定してください。

**diff**

<pre><code><div>   EasyVec(size_t n) {
     // メモリを確保し、配列の先頭を設定
     first = alloc.allocate(n);

     // 要素を初期化
     for (size_t i = 0; i < n; i++) {
       std::construct_at(first + i);
     }

     // 配列の終端を設定
     last = first + n;
   }
<font color="#3a3">+
+  // デストラクタ
+  ~EasyVec() {
+    // メモリを確保している場合のみ破棄と解放を実行
+    if (first) {
+
+      // 要素を破棄
+      for (iterator i = first; i != last; i++) {
+        std::destroy_at(i);
+      }
+
+      // メモリを解放
+      alloc.deallocate(first, last - first);
+    }
+  }
</font>
 private:
   T* first = nullptr; // 配列の先頭
   T* last = nullptr;  // 配列の終端
   std::allocator&lt;T&gt; alloc; // メモリ管理用のオブジェクト
 };
</div></code></pre>

デストラクタが実行された場合、その変数は削除されるので、以後はメンバ変数を読み書きすることはできません。<br>
そのため、`first`と`last`メンバ変数に`nullptr`を代入する、といった操作は不要です。

`EasyVec.h`にプログラムを追加し終わったら、以下の動作テストを実行してください。<br>
プログラムに問題がなければ`AC`が出力されます。


In [ ]:
# @title 動作テスト
!echo -e "#include \"EasyVec.h\"\nint main() {\nEasyVec<int> a;\nEasyVec<double> b(5);\n}\n">test.cpp
!g++ -otest -std=c++20 -Wall -Wextra test.cpp && ./test > /dev/null && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

### 1.7 添え字とサイズ

`EasyVec`では、`vector`ではおなじみの次のようなコードを書けるようにしたいです。

```cpp
EasyVec<int> v(5);
for (int i = 0; i < (int)v.size(); i++) {
  cin >> v[i];
}
```

これを可能にするには、`EasyVec`クラスに`size`メンバ関数と、`[]`演算子を定義する必要があります。

`size`メンバ関数は簡単なので、すぐに追加できます。`EasyVec.h`に次のプログラムを追加してください。

**diff**

<pre><code><div>   ~EasyVec() {
     // メモリを確保している場合のみ破棄と解放を実行
     if (first) {

       // 要素を破棄
       for (iterator i = first; i != last; i++) {
         std::destroy_at(i);
       }

       // メモリを解放
       alloc.deallocate(first, last - first);
     }
   }
<font color="#3a3">+
+  // サイズを取得
+  size_type size() const { return last - first; }
</font>
 private:
   T* first = nullptr; // 配列の先頭
   T* last = nullptr;  // 配列の終端
   std::allocator&lt;T&gt; alloc; // メモリ管理用のオブジェクト
 };
</div></code></pre>

配列のサイズは、「配列の終端の位置 - 配列の先頭の位置」で求められます。`size`メンバ関数はこの値を返すだけです。<br>
また、メンバ変数を書き換えないので、`const`メンバ関数としています。

`[]`演算子を定義するには、次のように書きます。

```cpp
戻り型 operator[](引数リスト) { プログラム }
```

`operator`(オペレータ)は「演算子」という意味です。関数名が`operator[]`となっているだけで、他の部分はメンバ関数と同じです。

それでは、`[]`演算子の「戻り型」、「引数」、「プログラム」を決めていきましょう。<br>
まず「戻り型」ですが、`cin >> v[i]`が有効なプログラムとなるには、`v[i]`が代入可能な値を返す必要があります。<br>
そして、その値は配列の要素でなくてはなりません。

代入については参照型を使えば実現できます。配列の要素は`value_type`型なので、戻り型は`value_type&`となります。

次に引数ですが、配列のどの位置でも指定できる「添え字」である必要があります。<br>
配列の大きさは`size_type`であらわせるので、添え字も`size_type`にしておくのが無難でしょう。

そして「プログラム」ですが、添え字を`i`とすると、`i`番目の要素を返すようになっていれば良さそうです。

ここまでの方針に従って`[]`演算子を追加しましょう。`EasyVec.h`に次のプログラムを追加してください。

**diff**

<pre><code><div>       // 要素を破棄
       for (iterator i = first; i != last; i++) {
         std::destroy_at(i);
       }

       // メモリを解放
       alloc.deallocate(first, last - first);
     }
   }

   // サイズを取得
   size_type size() const { return last - first; }
<font color="#3a3">+
+  // 添え字
+  value_type& operator[](size_type i) { return first[i]; }
+  const value_type& operator[](size_type i) const { return first[i]; }
</font>
 private:
   T* first = nullptr; // 配列の先頭
   T* last = nullptr;  // 配列の終端
   std::allocator&lt;T&gt; alloc; // メモリ管理用のオブジェクト
 };
</div></code></pre>

このプログラムでは、普通の`[]`演算子に加えて`const`メンバ関数版の`[]`演算子も定義しています。<br>
`const`メンバ関数版がないと、`EasyVec`クラスの`const`変数に対して`[]`演算子が使えないからです。

>メンバ関数の場合、`const`の有無でもオーバーロードできます。

`const`メンバ関数版の`[]`演算子は、戻り型を`const value_type&`としている点に注意してください。<br>
戻り型に`const`がないと、`const`な配列の値を書き換えることができてしまいます。<br>
それでは`const`変数の意味がなくなってしまうので、書き換えられないようにしなくてはならないのです。

また、「どうせ書き換えられないなら参照ではなく値を返すべき」という考え方もあります。<br>
ですが、もし値で返すとそれをコピーする時間が必要となります。参照ならばコピーは起きないので、CPUの時間を浪費せずに済みます。

`EasyVec.h`にプログラムを追加し終わったら、以下の動作テストを実行してください。<br>
プログラムに問題がなければ`AC`が出力されます。


In [ ]:
# @title 動作テスト
!echo -e "#include \"EasyVec.h\"\nint main() {\nEasyVec<int> a;\nEasyVec<double> b(5);\n}\n">test.cpp
!g++ -otest -std=c++20 -Wall -Wextra test.cpp && ./test > /dev/null && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

!echo -e "#include \"EasyVec.h\"\n#include <iostream>\nint main() {\nEasyVec<int> a;\nEasyVec<double> b(5);\nstd::cout << a.size() << std::endl;\nb[0]=4;b[1]=3;b[2]=2;b[1]+=1;\nfor(int i=0; i<(int)b.size(); i++) { cout << b[i] << " "; }\ncout << \"\n\";\n">test.cpp
!diff -Z <(echo -e "0\n4 3 2 1 0 ") <(g++ -otest -std=c++20 -Wall -Wextra test.cpp && ./test) > /dev/null && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

### 1.8 イテレータ

次に、イテレータを返す`begin`、`end`メンバ関数を定義しましょう。<br>
`EasyVec`の場合、これらの関数は、それぞれ`first`と`last`を返すだけです。

`EasyVec.h`に次のプログラムを追加してください。

**diff**

<pre><code><div>       // メモリを解放
       alloc.deallocate(first, last - first);
     }
   }

   // サイズの取得
   size_type size() const { return last - first; }

  // 添え字
   value_type& operator[](size_type i) { return first[i]; }
   const value_type& operator[](size_type i) const { return first[i]; }
<font color="#3a3">+
+  // イテレータ
+  iterator begin() { return first; }
+  const_iterator begin() const { return first; }
+  iterator end() { return last; }
+  const_iterator end() const { return last; }
</font>
 private:
   T* first = nullptr; // 配列の先頭
   T* last = nullptr;  // 配列の終端
   std::allocator&lt;T&gt; alloc; // メモリ管理用のオブジェクト
 };
</div></code></pre>

`[]`演算子の場合と同じ理由で、`begin`と`end`メンバ関数には`const`メンバ関数版も用意します。<br>
`const`メンバ関数版は、`const_iteretor`(コンスト・イテレータ)を返します。

`const_iterator`型の実態は`const T*`なので、これは「`const T`のポインタ」です。<br>
`const T`型の変数は、値を読むことはできても書き込みはできません。

`EasyVec.h`にプログラムを追加し終わったら、以下の動作テストを実行してください。<br>
プログラムに問題がなければ`AC`が出力されます。


In [ ]:
# @title 動作テスト
!echo -e "#include \"EasyVec.h\"\nint main() {\nEasyVec<int> a;\nEasyVec<double> b(5);\n}\n">test.cpp
!g++ -otest -std=c++20 -Wall -Wextra test.cpp && ./test > /dev/null && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

!echo -e "#include \"EasyVec.h\"\n#include <iostream>\nint main() {\nEasyVec<int> a;\nEasyVec<double> b(5);\nstd::cout << a.size() << std::endl;\nb[0]=4;b[1]=3;b[2]=2;b[1]+=1;\nfor(int i=0; i<(int)b.size(); i++) { cout << b[i] << " "; }\ncout << \"\n\";\n">test.cpp
!diff -Z <(echo -e "0\n4 3 2 1 0 ") <(g++ -otest -std=c++20 -Wall -Wextra test.cpp && ./test) > /dev/null && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

### 1.9 コピーコンストラクタ

クラスの変数を初期値に指定したり、`=`演算子で代入したりする操作は、クラスごとにプログラムすることができます。<br>
特にプログラムを書かなかった場合は、メンバ変数がすべてコピーされます。

それで十分な場合も多いのですが、残念なことに`EasyVec`クラスにとっては十分ではありません。<br>
例えば、次のようなプログラムを書いたとします。

```cpp
int main() {
  EasyVec<int> a;
  EasyVec<int> b = a;
}
```

このプログラムが実行されると、変数`a`と`b`のメンバ変数は、次の状態になります。

<img width="600px" hspace="50px" src="https://raw.githubusercontent.com/tn-mai/cpp2025/refs/heads/main/images/EasyVec_default_copy.drawio.png" />

実は、これは悪い状態です。その理由は「動的メモリ管理で取得したメモリを、2つのメンバ変数が指している」からです。<br>
この状態では、`b`を変更すると`a`も変更され、`a`を変更すると`b`も変更されることになります。

また、変数が削除されるときにも問題があります。例えば、`b`が先に削除されたとします。<br>
すると、`b.first`が指しているメモリは解放され、使えなくなります。ところが、`a.first`は使えなくなったメモリを指したままです。<br>
そのため、`a`が使えなくなったメモリを読み書きすると、論理エラーや実行時エラーが起こります。

問題はそれだけではありません。`a`が削除されるとき「解放済みのメモリ」をもう一度解放しようとします。<br>
これは、メモリの多重解放となり、実行時エラーが起こってしまうのです。

```cpp
int main() {
  EasyVec<int> a(5);
  {
    EasyVec<int> b = a; // firstとlastがコピーされる

    b[1] = 5; // b.first == a.first なので、a[1]を参照しても5になる

  } // ここでbが削除され、b.firstが指すメモリが解放される
    // b.first == a.first なので、a.firstは削除済みのメモリを指すことになる

  a[2] = 3; // NG. 解放済みのメモリに書き込んでいる

} // ここでaが削除される
  // 解放済みのメモリを再び解放しようとして、実行時エラーになる
```

これらの問題があるため、「すべてのメンバ変数をコピーする」という処理は、`EasyVec`クラスでは正しく機能しないのです。

そもそも、「配列をコピーしたい」というとき、本当にコピーしたいのは配列の内容です。<br>
しかし、動的に確保された配列の実体はポインタの指す先にあります。ポインタをコピーしても、配列はコピーされないのです。<br>
つまり、`EasyVec`クラスのコピー必要なのは、「メンバ変数ではなく配列の内容をコピーする」ことです。

<img width="550px" hspace="50px" src="https://raw.githubusercontent.com/tn-mai/cpp2025/refs/heads/main/images/EasyVec_allocate_and_copy.drawio.png" />

変数の初期値に「同じ型の値」を指定した場合、変数の初期化には「コピーコンストラクタ」が使われます。<br>
そこで、配列をコピーするプログラムはコピーコンストラクタに書きます。

>関数の引数が参照ではない場合も、コピーコンストラクタが実行されます。

`EasyVec.h`に次のプログラムを追加してください。

**diff**

<pre><code><div>   EasyVec(size_t n) {
     // メモリを確保し、配列の先頭を設定
     first = alloc.allocate(n);

     // 要素を初期化
     for (size_t i = 0; i < n; i++) {
       std::construct_at(first + i);
     }

     // 配列の終端を設定
     last = first + n;
   }
<font color="lightgreen">+
+  // コピーコンストラクタ
+  EasyVec(const EasyVec& other) {
+    // メモリを確保し、配列の先頭を設定
+    const size_t n = other.size();
+    first = alloc.allocate(n);
+
+    // 要素をコピーコンストラクタで初期化
+    for (size_t i = 0; i < n; i++) {
+      std::construct_at(first + i, other[i]);
+    }
+
+    // 配列の終端を設定
+    last = first + n;
+  }
</font>
   // デストラクタ
   ~EasyVec() {
     // メモリを確保している場合のみ破棄と解放を実行
     if (first) {

       // 要素を破棄
       for (iterator i = first; i != last; i++) {
         std::destroy_at(i);
       }
</div></code></pre>

メモリの確保と配列の終端の設定は、以前作成した引数付きコンストラクタと同じです。<br>
違いは`construct_at`関数に引数が追加された点だけです。<br>
`construct_at`関数の第2引数にコピーしたいオブジェクトを指定すると、コピーコンストラクタが呼び出されます。

`EasyVec.h`にプログラムを追加し終わったら、以下の動作テストを実行してください。<br>
プログラムに問題がなければ`AC`が出力されます。



In [ ]:
# @title 動作テスト
!echo -e "#include \"EasyVec.h\"\nint main() {\nEasyVec<int> a;\nEasyVec<double> b(5);\n}\n">test.cpp
!g++ -otest -std=c++20 -Wall -Wextra test.cpp && ./test > /dev/null && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

!echo -e "#include \"EasyVec.h\"\n#include <iostream>\nint main() {\nEasyVec<int> a;\nEasyVec<double> b(5);\nstd::cout << a.size() << std::endl;\nb[0]=4;b[1]=3;b[2]=2;b[1]+=1;\nfor(int i=0; i<(int)b.size(); i++) { cout << b[i] << " "; }\ncout << \"\n\";\n">test.cpp
!diff -Z <(echo -e "0\n4 3 2 1 0 ") <(g++ -otest -std=c++20 -Wall -Wextra test.cpp && ./test) > /dev/null && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

### 1.10 コピー代入演算子

変数のコピーを作成するとき、クラスの設計によってはコピーコンストラクタを書く必要がありました。

これは、「代入」でも同じことです。例えば`EasyVec`クラスの場合、本当にコピーしたいのは`first`メンバ変数が指す配列の中身で、`first`や`last`メンバ変数ではありません。

さて、代入は、次のようなプログラムで発生します。

```cpp
EasyVec<int> a;
EasyVec<int> b;
b = a; // 代入
```

C++では、クラス専用の代入演算子を定義できます。<br>
代入演算子を定義するには、次のように書きます。

```cpp
戻り型 operator=(引数リスト) { プログラム }
```

気がついたかもしれませんが、記号が異なる以外は`[]`演算子と同じです。

それでは、代入演算子の「戻り型」、「引数」、「プログラム」を決めていきましょう。<br>
まず「戻り型」ですが、代入演算子の場合は「自分自身の参照を返すべき」という暗黙のルールがあります。<br>
これは、次のようなプログラムを書けるようにするためです。

```cpp
EasyVec<int> a, b, c;
a = b = c; // a = (b = c)という意味
```

もし`b = c`という式が`EasyVec`型の値を返さないと、`a = (b = c)`が成立しません。<br>
そのため、戻り型は`EasyVec&`でなくてはならないのです。

次に引数ですが、今回は「同じ型の値をコピーしたい」ので、`const EasyVec&`とします。<br>
`const`が付いているのは、「コピー元を変更しないことを保証する」ためです。

>このような「同じ型の値をコピーする代入演算子」は、「コピー代入演算子」と呼ばれます。

最後に「プログラム」ですが、以下の2点に注意する必要があります。

1. コピー先とコピー元が同じオブジェクトの場合
2. コピー先とコピー元で、配列のサイズが異なる場合

1番目の問題は簡単です。同じオブジェクトの場合は「何もしなければよい」のです。<br>
2番目の問題は少し難しいです。サイズが等しい場合はただコピーするだけで済みます。<br>
ですが、サイズが異なる場合は、コピー先の配列のメモリを解放して、作り直さなくてはなりません。


#### 配列の初期化と破棄を関数にする

現在、配列の作成と破棄(はき)を行うプログラムは、コンストラクタとデストラクタに直接書いてあります。<br>
配列を作り直すには、コピー代入演算子にもこれらと同じプログラムを書かなくてはなりません。

ですが、同じプログラムを何度も書くのは非効率です。何度も使うプログラムは、関数にしておくべきです。

配列を作成する関数の名前は`create`(クリエイト)、配列を破棄する関数の名前は`destroy`(デストロイ)としましょう。<br>
また、これらの関数はクラスの内部での利用を想定しており、外部から呼び出されると整合性が取れなくなる可能性があります。<br>
このようなメンバは「インターフェイスではない」ので、`private`メンバにします。

関数の中身はあとにして、とりあえず定義だけしてしまいましょう。`EasyVec.h`に次のプログラムを追加してください。

**diff**

<pre><code><div>   iterator end() { return last; }
   const_iterator end() const { return last; }

 private:
<font color="#3a3">+  // 配列を作成する
+  void create(const EasyVec& other) {
+  }
+
+  // 配列を破棄する
+  void destroy() {
+    }
+  }
+</font>
   T* first = nullptr; // 配列の先頭
   T* last = nullptr;  // 配列の終端
   std::allocator&lt;T&gt; alloc; // メモリ管理用のオブジェクト
 };
</div></code></pre>

次に、配列を作成するプログラムを、コピーコンストラクタから`create`メンバ関数に移動させます。<br>
まず、コピーコンストラクタに`create`メンバ関数の呼び出しを追加します。`EasyVec.h`に次のプログラムを追加してください。

**diff**

<pre><code><div>     // 要素を初期化
     for (size_t i = 0; i < n; i++) {
       std::construct_at(first + i);
     }

     // 配列の終端を設定
     last = first + n;
   }

   // コピーコンストラクタ
   EasyVec(const EasyVec& other) {
<font color="#3a3">+    create(other);</font>
     // メモリを確保し、配列の先頭を設定
     const size_t n = other.size();
     first = alloc.allocate(n);

     // 要素をコピーコンストラクタで初期化
     for (size_t i = 0; i < n; i++) {
       std::construct_at(first + i, other[i]);
     }
</pre></code></div>

次に、コピーコンストラクタのプログラムを、`create`メンバ関数に移動させます。<br>
移動には`Ctrl+X`(切り取り)と`Ctrl+V`(貼り付け)を使います。<br>
`EasyVec.h`のコピーコンストラクタのプログラムを範囲選択し、`Ctrl+X`で切り取ってください。

>**【diffの-(マイナス)記号について】**<br>
> 以下のdiffで、先頭に`-`(マイナス)記号が付いている行は「元のプログラムから削除する行」という意味になります。<br>
> 例えば、次のdiffの場合、
>
> <pre><code><div>   // コピーコンストラクタ
>    EasyVec(const EasyVec& other) {
>      create(other);
> <font color="#e33">-    // メモリを確保し、配列の先頭を設定
> -    const size_t n = other.size();
> -    first = alloc.allocate(n);
> -
> -    // 要素をコピーコンストラクタで初期化
> -    for (size_t i = 0; i < n; i++) {
> -      std::construct_at(first + i, other[i]);
> -    }
> -
> -    // 配列の終端を設定
> -    last = first + n;</font>
>    }
> </div></code></pre>
>
> 先頭が`-`の行を削除して、次のように変更します。
>
> <pre><code><div>   EasyVec(const EasyVec& other) {
>      create(other);
>    }
> </div></code></pre>
>
> 単に削除するだけの場合もあれば、今回のように`Ctrl+X`で切り取る場合もあります。

**diff**

<pre><code><div>     // 要素を初期化
     for (size_t i = 0; i < n; i++) {
       std::construct_at(first + i);
     }

     // 配列の終端を設定
     last = first + n;
   }

   // コピーコンストラクタ
   EasyVec(const EasyVec& other) {
     create(other);
<font color="#e33">-    // メモリを確保し、配列の先頭を設定
-    const size_t n = other.size();
-    first = alloc.allocate(n);
-
-    // 要素をコピーコンストラクタで初期化
-    for (size_t i = 0; i < n; i++) {
-      std::construct_at(first + i, other[i]);
-    }
-
-    // 配列の終端を設定
-    last = first + n;</font>
   }

   // デストラクタ
   ~EasyVec() {
     // メモリを確保している場合のみ破棄と解放を実行
     if (first) {
</div></code></pre>

次に、切り取ったプログラムを、`create`メンバ関数の中に`Ctrl+X`で貼り付けてください。

**diff**

<pre><code><div>
   iterator end() { return last; }
   const_iterator end() const { return last; }

 private:
   // 配列を作成する
   void create(const EasyVec& other) {
<font color="#3a3">+    // メモリを確保し、配列の先頭を設定
+    const size_t n = other.size();
+    first = alloc.allocate(n);
+
+    // 要素をコピーコンストラクタで初期化
+    for (size_t i = 0; i < n; i++) {
+      std::construct_at(first + i, other[i]);
+    }
+
+    // 配列の終端を設定
+    last = first + n;</font>
   }

   // 配列を破棄する
   void destroy() {
   }
</pre></code></div>

同じ手順で、`EasyVec.h`のデストラクタのプログラムを切り取り、`destroy`メンバ関数に貼り付けてください。

**diff**

<pre><code><div>   EasyVec(const EasyVec& other) {
     create(other);
   }

   // デストラクタ
   ~EasyVec() {
<font color="#3a3">+    destroy()</font>
<font color="#e33">-    // メモリを確保している場合のみ破棄と解放を実行
-    if (first) {
-
-      // 要素を破棄
-      for (iterator i = first; i != last; i++) {
-        std::destroy_at(i);
-      }
-
-      // メモリを解放
-      alloc.deallocate(first, last - first);
-    }</font>
   }

   // サイズを取得
   size_type size() const { return last - first; }

   // 添え字
   value_type& operator[](size_type i) { return first[i]; }
   const value_type& operator[](size_type i) const { return first[i]; }

   // イテレータ
   iterator begin() { return first; }
   const_iterator begin() const { return first; }
   iterator end() { return last; }
   const_iterator end() const { return last; }

 private:
   // 配列を作成する
   void create(const EasyVec& other) {
     // メモリを確保し、配列の先頭を設定
     const size_t n = other.size();
     first = alloc.allocate(n);

     // 要素をコピーコンストラクタで初期化
     for (size_t i = 0; i < n; i++) {
       std::construct_at(first + i, other[i]);
     }

     // 配列の終端を設定
     last = first + n;
   }

   // 配列を破棄する
   void destroy() {
<font color="#3a3">+    // メモリを確保している場合のみ破棄と解放を実行
+    if (first) {
+
+      // 要素を破棄
+      for (iterator i = first; i != last; i++) {
+        std::destroy_at(i);
+      }
+
+      // メモリを解放
+      alloc.deallocate(first, last - first);
+
+      // ポインタを初期化
+      first = last = nullptr;
+    }</font>
   }

   T* first = nullptr; // 配列の先頭
   T* last = nullptr;  // 配列の終端
   std::allocator&lt;T&gt; alloc; // メモリ管理用のオブジェクト
 };
</div></code></pre>

デストラクタと違って、`destroy`メンバ関数は実行後もオブジェクトを読み書きできる可能性があります。<br>
安全のために、`first`と`last`を初期化しておきます。<br>
`EasyVec.h`に次のプログラムを追加してください。

**diff**

<pre><code><div>   // 配列を破棄する
   void destroy() {
     // メモリを確保している場合のみ破棄と解放を実行
     if (first) {

       // 要素を破棄
       for (iterator i = first; i != last; i++) {
         std::destroy_at(i);
       }

       // メモリを解放
       alloc.deallocate(first, last - first);
<font color="#3a3">+
+      // ポインタを初期化
+      first = last = nullptr;</font>
     }
   }

   T* first = nullptr; // 配列の先頭
   T* last = nullptr;  // 配列の終端
   std::allocator&lt;T&gt; alloc; // メモリ管理用のオブジェクト
 };
</div></code></pre>

`EasyVec.h`の変更が終わったら、以下の動作テストを実行してください。<br>
プログラムに問題がなければACが出力されます。


In [ ]:
# @title 動作テスト
!echo -e "#include \"EasyVec.h\"\nint main() {\nEasyVec<int> a;\nEasyVec<double> b(5);\n}\n">test.cpp
!g++ -otest -std=c++20 -Wall -Wextra test.cpp && ./test > /dev/null && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

!echo -e "#include \"EasyVec.h\"\n#include <iostream>\nint main() {\nEasyVec<int> a;\nEasyVec<double> b(5);\nstd::cout << a.size() << std::endl;\nb[0]=4;b[1]=3;b[2]=2;b[1]+=1;\nfor(int i=0; i<(int)b.size(); i++) { cout << b[i] << " "; }\ncout << \"\n\";\n">test.cpp
!diff -Z <(echo -e "0\n4 3 2 1 0 ") <(g++ -otest -std=c++20 -Wall -Wextra test.cpp && ./test) > /dev/null && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

#### コピー代入演算子を定義する

追加した`create`メンバ関数と`destroy`メンバ関数を使って、代入演算子を定義しましょう。<br>
`EasyVec.h`に次のプログラムを追加してください。

**diff**

<pre><code><div>   EasyVec(const EasyVec& other) {
     create(other);
   }

   // デストラクタ
   ~EasyVec() {
     destroy();
   }
<font color="#3a3">+
+  // コピー代入演算子
+  EasyVec& operator=(const EasyVec& other) {
+    // コピー元とコピー先が異なる場合だけコピーを実行
+    if (this != &other) {
+
+      if (size() != other.size()) {
+        // サイズが異なる場合は配列を作り直す
+        destroy();
+        create(other);
+      } else {
+        // サイズが等しい場合はデータをコピーするだけ
+        for (int i = 0; i < (int)size(); i++) {
+          first[i] = other[i];
+        }
+      }
+    } // if *this != other
+
+    return *this; // このオブジェクトの参照を返す
+  }
</font>
   size_type size() const { return last - first; }

   value_type& operator[](size_type i) { return first[i]; }
   const value_type& operator[](size_type i) const { return first[i]; }
</div></code></pre>

「コピー先とコピー元が同じ場合」を検出するには、コピー元である`other`のアドレスと、コピー先のアドレスである`this`を比較します。<br>
`&other`と`this`が同じアドレスを指している場合、コピー元とコピー先は同じオブジェクトだと判定できます。

「コピー元とコピー先が等しい場合」を「自己代入(じこだいにゅう)」といいます。自己代入の場合は何もせずに`*this`を返します。

自己代入ではなかった場合は、配列をコピーします。コピー元とコピー先のサイズが等しい場合はコピーするだけです。<br>
ですが、サイズが異なる場合は、コピー先の配列がコピー元と同じサイズとなるように作り直さなくてはなりません。

さて、長くなりましたが、これでコピー代入演算子は完成です。<br>
コピー代入演算子を定義したことで、変数の代入を適切に処理できるようになりました。

`EasyVec.h`の変更が終わったら、以下の動作テストを実行してください。<br>
プログラムに問題がなければACが出力されます。


In [ ]:
# @title 動作テスト
!echo -e "#include \"EasyVec.h\"\nint main() {\nEasyVec<int> a;\nEasyVec<double> b(5);\n}\n">test.cpp
!g++ -otest -std=c++20 -Wall -Wextra test.cpp && ./test > /dev/null && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

!echo -e "#include \"EasyVec.h\"\n#include <iostream>\nint main() {\nEasyVec<int> a;\nEasyVec<double> b(5);\nstd::cout << a.size() << std::endl;\nb[0]=4;b[1]=3;b[2]=2;b[1]+=1;\nfor(int i=0; i<(int)b.size(); i++) { cout << b[i] << " "; }\ncout << \"\n\";\n">test.cpp
!diff -Z <(echo -e "0\n4 3 2 1 0 ") <(g++ -otest -std=c++20 -Wall -Wextra test.cpp && ./test) > /dev/null && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

### 1.11 代入と初期化を区別する

代入と初期化は、どちらも`=`記号を使うことから、同じだと勘違いされがちです。<br>
ですが、同じ記号を使っていても、この2つは全く違うものです。

`=`が初期化に使われるときは「コピーコンストラクタ」が呼び出されますが、`=`が代入に使われるときは「コピー代入演算子」が呼び出されるからです。実際の挙動でも、コピー代入演算子は「すでにあるデータを上書きする」のに対して、コピーコンストラクタは「なにもない場所に書き込む」という違いがあります。

初期化が行われるのは次のような場面です。

* 変数の宣言
* 変数の初期化
* 関数呼び出しの引数に変数を渡す

これに対して、代入が行われるのは次の場面だけです。

* 既存の変数に別の変数を代入する

上記の違いをプログラムであらわすと、次のようになります。

```cpp
EasyVec<int> a;     // 初期化(デフォルトコンストラクタを呼び出す)
EasyVec<int> b(7);  // 初期化(引数付きコンストラクタを呼び出す)
EasyVec<int> c(a);  // 初期化(コピーコンストラクタを呼び出す)
EasyVec<int> d = b; // 初期化(コピーコンストラクタを呼び出す)

a = b;         // 代入(コピー代入演算子を呼び出す)
```

変数宣言の丸括弧の中に、同じ型の変数を指定すると、コピーコンストラクタが呼び出される点に注意してください。<br>
つまり、`EasyVec c(a);`と、`EasyVec c = a;`は同じ意味になります。

初期化と代入は、それぞれ異なる操作です。この2つを区別することは重要です。

> コンストラクタは常に初期化をコントロールする。<br>
> コピー代入演算子は常に代入をコントロールする。


### 1.12 push_backとpop_back


#### 配列のサイズを増やしたり減らしたりする効率的な方法

`EasyVec`クラスは、ここまでに追加したプログラムによって、配列クラスとしてある程度使えるようになっています。<br>
しかし、お手本の`vector`にできて`EasyVec`にはできないことはまだまだあります。

例えば、お手本にした`vector`クラスの場合、`push_back`メンバ関数や`pop_back`メンバ関数によってサイズを変えられます。<br>
ところが、現在の`EasyVec`クラスのサイズは、コンストラクタで決めた値から変えられません。

`push_back`メンバ関数を付け加えるとしたら、たとえばサイズを1つ大きくした別のメモリを確保し、現在の配列をすべてコピーして、`first`が新しいメモリを指すように書き換える、という方法が考えられます。

しかし、この方法では`push_back`を行うたびにメモリの確保とコピーが行われるため、CPUの時間をかなり浪費してしまいます。

それでは、`vector`クラスはどのようにこの問題を解決しているのかという疑問が浮かびます。<br>
実は`vector`クラスは「最初から余分なメモリを確保しておく」という方法を使っています。<br>
そして、配列が大きくなってメモリが足りなくなったときに限り、より多くのメモリを確保しなおします。

この方法では、「現在の配列の終端」に加えて、「確保したメモリ領域の終端」をあらわすメンバ変数が必要となります。<br>
このメンバ変数の名前を`cap`(キャップ、`capacity`の短縮形)とすると、データ構造は次のようになるでしょう。

<img width="650px" hspace="50px" src="https://raw.githubusercontent.com/tn-mai/cpp2025/refs/heads/main/images/EasyVec_capacity.drawio.png" />


#### メモリの終端を指すメンバ変数を追加する

とりあえず、`cap`メンバ変数を追加しましょう。`EasyVec.h`に次のプログラムを追加してください。

**diff**

<pre><code><div>       // メモリを解放
       alloc.deallocate(first, last - first);

       // ポインタを初期化
       first = last = nullptr;
     }
   }

   T* first = nullptr; // 配列の先頭
   T* last = nullptr;  // 配列の終端
<font color="#3a3">+  T* cap = nullptr;   // 確保したメモリ領域の終端</font>
   std::allocator&lt;T&gt; alloc; // メモリ管理用のオブジェクト
 };
</div></code></pre>

`cap`メンバ変数には、配列を作成するときに「確保したメモリ領域の終端」を設定します。配列の作成は`create`メンバ関数が行います。<br>
現在の`create`メンバ関数では、`last`だけに「確保したメモリ領域の終端」を代入しています。<br>
同じ値を`cap`にも代入すれば、`cap`が適切な位置を指すようになるはずです。

`EasyVec.h`の`create`メンバ関数を次のように変更してください。

>diffで「行の変更」をあらわすには、`-`で元の行を削除し、`+`で変更後の行を追加する、という書きかたになります。<br>
>このような変更を見つけたら、違いを見比べて「変更されている部分だけ」書き換えてください。<br>
>例えば、以下のdiffの場合、
>
><pre><code><div><font color="#e33">-    last = first + n;</font>
><font color="#3a3">+    last = cap = first + n;</font></div></code></pre>
>
>実際にやることは、次のように`last`と`= first + n;`のあいだに、`= cap`を追加しすることです。
>
><pre><code><div>  last<font color="#3a3"> = cap</font> = first + n;</div></code></pre>
>
>なお、この例では追加するだけでしたが、diffの内容によっては「一部を削除する」必要があります。

**diff**

<pre><code><div>   // 配列を作成する
   void create(const EasyVec& other) {
     // メモリを確保し、配列の先頭を設定
     const size_t n = other.size();
     first = alloc.allocate(n);

     // 要素をコピーコンストラクタで初期化
     for (size_t i = 0; i < n; i++) {
       std::construct_at(first + i, other[i]);
     }

     // 配列の終端を設定
<font color="#e33">-    last = first + n;</font>
<font color="#3a3">+    last = cap = first + n;</font>
   }

   void destroy() {
     // メモリを確保している場合のみ破棄と解放を実行
     if (first) {
</div></code></pre>

それから、配列を破棄したら、`cap`には`nullptr`を設定して、メモリを確保していないことを示す必要があります。<br>
`EasyVec.h`の`destroy`メンバ関数を次のように変更してください。

**diff**

<pre><code><div>   // 配列を破棄する
   void destroy() {
     // メモリを確保している場合のみ破棄と解放を実行
     if (first) {

       // 要素を破棄
       for (iterator i = first; i != last; i++) {
         std::destroy_at(i);
       }

       // メモリを解放
       alloc.deallocate(first, last - first);

       // ポインタを初期化
<font color="#e33">-      first = last = nullptr;</font>
<font color="#3a3">+      first = last = cap = nullptr;</font>
     }
   }

   T* first = nullptr; // 配列の先頭
   T* last = nullptr;  // 配列の終端
   T* cap = nullptr;   // 確保したメモリ領域の終端
   std::allocator&lt;T&gt; alloc; // メモリ管理用のオブジェクト
 };
</div></code></pre>

これで、既存のメンバ関数を`cap`メンバ変数に対応させることができました。

`EasyVec.h`の変更が終わったら、以下の動作テストを実行してください。<br>
プログラムに問題がなければ`AC`が出力されます。


In [ ]:
# @title 動作テスト
!echo -e "#include \"EasyVec.h\"\nint main() {\nEasyVec<int> a;\nEasyVec<double> b(5);\n}\n">test.cpp
!g++ -otest -std=c++20 -Wall -Wextra test.cpp && ./test > /dev/null && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

!echo -e "#include \"EasyVec.h\"\n#include <iostream>\nint main() {\nEasyVec<int> a;\nEasyVec<double> b(5);\nstd::cout << a.size() << std::endl;\nb[0]=4;b[1]=3;b[2]=2;b[1]+=1;\nfor(int i=0; i<(int)b.size(); i++) { cout << b[i] << " "; }\ncout << \"\n\";\n">test.cpp
!diff -Z <(echo -e "0\n4 3 2 1 0 ") <(g++ -otest -std=c++20 -Wall -Wextra test.cpp && ./test) > /dev/null && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

#### 余分なメモリを確保するメンバ関数を追加する

続いて、配列のメモリを使い切ったとき、余分なメモリを確保する関数を作成しましょう。<br>
関数名は`grow`(グロー、「育つ、成長する」という意味)とします。

ところで、一度確保したメモリ領域は、サイズを小さくすることは簡単ですが、大きくすることは難しいです。

メモリの確保というのは、メモ帳に喩えると「メモ帳のあるページから指定のページ数までの範囲を予約した状態」だからです。<br>
ここからサイズを小さくするのは簡単で、後ろのページの割り当てを解除するだけです。

それに対して、サイズを大きくする場合は、予約したページの末尾に、連続した未使用ページが必要になます。<br>
ですが、後ろのページは既に別の用途で予約済みなことが多く、十分な未使用ページがあることは稀です。

$
\begin{array}{|c|c|c|} \hline
\dots & EasyVec用のメモリ　　　　 & int \space number & \dots \\ \hline
\end{array} \\
　　　　　　　　　　　　　↑この後ろのメモリを使いたいが…
$

そのため、サイズを大きくしたい場合は、別のページを新しく予約しなおすことになります。

当然ですが、新しく予約したページには何も書かれていないので、元のページからデータをコピーしてくる必要があります。

つまり、`grow`メンバ関数は、まず余分を考慮したサイズの新しいメモリを確保し、そのメモリに配列をコピーする必要があります。<br>
このことを考慮した`grow`メンバ関数のやるべきことは、次の手順になるでしょう。

1. 確保するメモリのサイズを計算
2. 新しいメモリを確保し、一時変数にアドレスを保存
2. 配列のすべてのデータを、新しいメモリにコピー
3. 元の配列を削除
4. 新しいメモリのアドレスとサイズをメンバ変数に反映

また、`grow`メンバ関数はインターフェイスではないので、プライベートメンバにします。<br>
それでは、`EasyVec.h`に`grow`メンバ関数の定義を追加してください。

**diff**

<pre><code><div>       // メモリを解放
       alloc.deallocate(first, last - first);

       // ポインタを初期化
       first = last = cap = nullptr;
     }
   }
<font color="#3a3">+
+  // 余分なメモリを確保する
+  void grow() {
+    // 確保するメモリのサイズを計算
+    size_t n = 1; // メモリが確保されていない場合のサイズは1とする
+    if (first) {
+      // メモリが確保されている場合、元の2倍のメモリを確保する
+      n = (cap - first) * 2;
+    }
+
+    // 新しいメモリを確保
+    T* p = alloc.allocate(n);
+
+    // 配列のすべてのデータを新しいメモリにコピー
+    const size_t m = size();
+    for (size_t i = 0; i < m; i++) {
+      std::construct_at(p + i, first[i]);
+    }
+
+    // 元の配列を削除
+    destroy();
+
+    // 新しいメモリのアドレスとサイズをメンバ変数に反映
+    first = p;
+    last = p + m;
+    cap = p + n;
+  }</font>

   T* first = nullptr; // 配列の先頭
   T* last = nullptr;  // 配列の終端
   T* cap = nullptr;   // 確保したメモリ領域の終端
   std::allocator&lt;T&gt; alloc; // メモリ管理用のオブジェクト
 };
</div></code></pre>

`EasyVec.h`の変更が終わったら、以下の動作テストを実行してください。<br>
プログラムに問題がなければ`AC`が出力されます。


In [ ]:
# @title 動作テスト
!echo -e "#include \"EasyVec.h\"\nint main() {\nEasyVec<int> a;\nEasyVec<double> b(5);\n}\n">test.cpp
!g++ -otest -std=c++20 -Wall -Wextra test.cpp && ./test > /dev/null && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

!echo -e "#include \"EasyVec.h\"\n#include <iostream>\nint main() {\nEasyVec<int> a;\nEasyVec<double> b(5);\nstd::cout << a.size() << std::endl;\nb[0]=4;b[1]=3;b[2]=2;b[1]+=1;\nfor(int i=0; i<(int)b.size(); i++) { cout << b[i] << " "; }\ncout << \"\n\";\n">test.cpp
!diff -Z <(echo -e "0\n4 3 2 1 0 ") <(g++ -otest -std=c++20 -Wall -Wextra test.cpp && ./test) > /dev/null && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

#### push_backメンバ関数を定義する

`push_back`メンバ関数の機能は「配列の末尾に要素をひとつ追加する」ことです。<br>
確保したメモリに未使用の部分が残っていれば、そこに要素をコピーし、サイズを1増やします。<br>
未使用の部分が残っていない場合は、要素をコピーする前に、`grow`メンバ関数を使ってメモリサイズを増やしておく必要があります。

`EasyVec.h`に、`push_back`メンバ関数の定義を追加してください。

**diff**

<pre><code><div>   // イテレータ
   iterator begin() { return first; }
   const_iterator begin() const { return first; }
   iterator end() { return last; }
   const_iterator end() const { return last; }
<font color="#3a3">+
+  // 末尾に要素を追加する
+  void push_back(const T& x) {
+    // 未使用の部分が残っていなければ、配列のサイズを増やす
+    if (last == cap) {
+      grow();
+    }
+
+    // 配列の終端に要素を追加する
+    std::construct_at(last, x);
+
+    // 配列のサイズをひとつ増やす
+    last++;
+  }
</font>
 private:
   // 配列を作成する
   void create(const EasyVec& other) {
     // メモリを確保し、配列の先頭を設定
     const size_t n = other.size();
     first = alloc.allocate(n);
</div></code></pre>

`EasyVec.h`の変更が終わったら、以下の動作テストを実行してください。<br>
プログラムに問題がなければ`AC`が出力されます。


In [ ]:
# @title 動作テスト
!echo -e "#include \"EasyVec.h\"\nint main() {\nEasyVec<int> a;\nEasyVec<double> b(5);\n}\n">test.cpp
!g++ -otest -std=c++20 -Wall -Wextra test.cpp && ./test > /dev/null && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

!echo -e "#include \"EasyVec.h\"\n#include <iostream>\nint main() {\nEasyVec<int> a;\nEasyVec<double> b(5);\nstd::cout << a.size() << std::endl;\nb[0]=4;b[1]=3;b[2]=2;b[1]+=1;\nfor(int i=0; i<(int)b.size(); i++) { cout << b[i] << " "; }\ncout << \"\n\";\n">test.cpp
!diff -Z <(echo -e "0\n4 3 2 1 0 ") <(g++ -otest -std=c++20 -Wall -Wextra test.cpp && ./test) > /dev/null && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

#### pop_backメンバ関数を定義する

`push_back`メンバ関数と比べると、`pop_back`メンバ関数はかなりシンプルです。

`pop_back`メンバ関数がやることは、配列の末尾の要素を削除して、サイズをひとつ減らすことです。<br>
ただし、サイズが0の場合は何もしません。

それから、配列のサイズを減らす場合は、確保したメモリを減らす処理は書きません。<br>
あとでデータが追加されて、結局メモリが必要になる可能性が高いからです。

それでは、`EasyVec.h`に、`pop_back`メンバ関数の定義を追加してください。

**diff**

<pre><code><div>   // 末尾に要素を追加する
   void push_back(const T& x) {
     // 未使用の部分が残っていなければ、配列のサイズを増やす
     if (last == cap) {
       grow();
     }

     // 配列の終端に要素を追加する
     std::construct_at(last, x);

     // 配列のサイズをひとつ増やす
     last++;
   }
<font color="#3a3">+
+  // 末尾の要素を削除する
+  void pop_back() {
+    // サイズが0でなければ要素をひとつ減らす
+    if (size()) {
+      std::destroy_at(last - 1);
+      last--;
+    }
+  }
</font>
 private:
   // 配列を作成する
   void create(const EasyVec& other) {
     // メモリを確保し、配列の先頭を設定
     const size_t n = other.size();
     first = alloc.allocate(n);
</div></code></pre>

`last`メンバ変数が指すのは配列の終端で、これは「最後の要素の次の位置」になります。<br>
そのため、最後の要素を参照するには`last - 1`とする必要があります。

`EasyVec.h`の変更が終わったら、以下の動作テストを実行してください。<br>
プログラムに問題がなければ`AC`が出力されます。


In [ ]:
# @title 動作テスト
!echo -e "#include \"EasyVec.h\"\nint main() {\nEasyVec<int> a;\nEasyVec<double> b(5);\n}\n">test.cpp
!g++ -otest -std=c++20 -Wall -Wextra test.cpp && ./test > /dev/null && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

!echo -e "#include \"EasyVec.h\"\n#include <iostream>\nint main() {\nEasyVec<int> a;\nEasyVec<double> b(5);\nstd::cout << a.size() << std::endl;\nb[0]=4;b[1]=3;b[2]=2;b[1]+=1;\nfor(int i=0; i<(int)b.size(); i++) { cout << b[i] << " "; }\ncout << \"\n\";\n">test.cpp
!diff -Z <(echo -e "0\n4 3 2 1 0 ") <(g++ -otest -std=c++20 -Wall -Wextra test.cpp && ./test) > /dev/null && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

### 1.13 データの消去

最初のインターフェイスには書きませんでしたが、すべてのデータを消去したい、という要求は比較的多いです。<br>
`vector`クラスの場合、これは`clear`(クリア)メンバ関数の機能です。

`clear`メンバ関数は、作成済みのすべての要素について`destroy_at`関数を呼び出して要素を削除し、その後サイズを`0`にします。

`EasyVec.h`に、`clear`メンバ関数の定義を追加してください。

**diff**

<pre><code><div>   // 末尾の要素を削除する
   void pop_back() {
     // サイズが0でなければ要素をひとつ減らす
     if (size()) {
       std::destroy_at(last - 1);
       last--;
     }
   }
<font color="#3a3">+
+  // すべての要素を削除する
+  void clear() {
+    for (size_t i = 0; i < size(); i++) {
+      std::destroy_at(first + i);
+    }
+    last = first;
+  }
</font>
 private:
   // 配列を作成する
   void create(const EasyVec& other) {
     // メモリを確保し、配列の先頭を設定
     const size_t n = other.size();
     first = alloc.allocate(n);
</div></code></pre>

`EasyVec.h`の変更が終わったら、以下の動作テストを実行してください。<br>
プログラムに問題がなければ`AC`が出力されます。




In [ ]:
# @title 動作テスト
!echo -e "#include \"EasyVec.h\"\nint main() {\nEasyVec<int> a;\nEasyVec<double> b(5);\n}\n">test.cpp
!g++ -otest -std=c++20 -Wall -Wextra test.cpp && ./test > /dev/null && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

!echo -e "#include \"EasyVec.h\"\n#include <iostream>\nint main() {\nEasyVec<int> a;\nEasyVec<double> b(5);\nstd::cout << a.size() << std::endl;\nb[0]=4;b[1]=3;b[2]=2;b[1]+=1;\nfor(int i=0; i<(int)b.size(); i++) { cout << b[i] << " "; }\ncout << \"\n\";\n">test.cpp
!diff -Z <(echo -e "0\n4 3 2 1 0 ") <(g++ -otest -std=c++20 -Wall -Wextra test.cpp && ./test) > /dev/null && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

### 1.14 explicit(エクスプリシット)キーワード

整数型の計算では、異なる型を混ぜて式を作ることができます。

```cpp
double a = 10; // int型の10をdouble型の10.0に変換し、aを10.0で初期化

double b;
b = 10; // int型の10をdouble型の10.0に変換し、bに10.0を代入

string s = "abc"; // const char[4]をconst char*として受け取り、sを初期化
```

このように、型が違っても代入できるのは、C++には「暗黙の型変換(あんもくのかたへんかん)」という機能があるからです。

暗黙の型変換は、`int`などの組み込み型だけでなく、プログラマが定義したクラスでも利用可能です。<br>
作成したクラスに「引数がひとつだけのコンストラクタ」がある場合、引数の型からの暗黙の型変換が有効になります。

そして、`EasyVec`クラスには、「`size_t`型の引数がひとつだけのコンストラクタ」があります。<br>
このため、以下のようなプログラムを書くことができてしまいます。

```cpp
int total(const EasyVec<int>& v) {
  int a = 0;
  for (auto i = v.begin(); i != v.end(); i++) {
    a += *i;
  }
  return a;
}

double two(double d) { return d * d; }

int main() {
  EasyVec<int> v(10);
  v = 10; // v[0] = 10;の間違い？
  double x = total(3.14); // two(3.14)の間違い？
}
```

この例では、配列に`int`型の`10`を代入したり、合計を求める`total`関数に`double`型の`3.14`を渡したりしています。<br>
コメントにあるように、これらは書き間違いの可能性が高いです。しかし、エラーにはなりません。<br>
なぜなら、暗黙の型変換によって次のように書き換えられてしまうからです。

```cpp
int main() {
  EasyVec<int> v(10);
  v = EasyVec(10); // 暗黙の型変換で10がEasyVec(10)になる
  double x = total(EasyVec(3)); // 暗黙の型変換で3.14がEasyVec(3)になる
}
```

このように、暗黙の型変換には「間違ったプログラム」につながる危険性があります。<br>
そのため、場合によっては「暗黙の型変換を禁止」できると便利です。そしてもちろん、禁止することは可能です。

暗黙の型変換を禁止するには、コンストラクタの前に`explicit`(エクスプリシット)キーワードを書きます。

`EasyVec.h`に次のプログラムを追加してください。

**diff**

<pre><code><div>   // 型の定義
   using iterator = T*;
   using const_iterator = const T*;
   using size_type = size_t;
   using value_type = T;

   // コンストラクタ
   EasyVec() = default;
<font color="#e33">-  EasyVec(size_t n) {</font>
<font color="#3a3">+  explicit EasyVec(size_t n) {</font>
     // メモリを確保し、配列の先頭を設定
     first = alloc.allocate(n);

     // 要素を初期化
     for (size_t i = 0; i < n; i++) {
       std::construct_at(first + i);
     }
</div></code></pre>

`explicit`キーワードを追加したことで、前に見たプログラムはエラーになります。

```cpp
int main() {
  EasyVec<int> v(10);
  v = 10; // エラー. 暗黙の型変換が禁止されている
  double x = total(3.14); // エラー. 暗黙の型変換が禁止されている
}
```

暗黙の型変換は役に立つこともあります。<br>
ですが、`EasyVec`クラスのように、暗黙の型変換を禁止したほうが良い場合も多いのです。


### 1.15 インクルード・ガード

C++の「インクルード」という仕組みは、「指定されたファイルの内容を、インクルード指令の位置にコピーする」というものです。<br>
そのため、同じファイルを2回以上インクルードすると、ファイルも2回以上コピーされます。

しかし、同じファイルが2回以上コピーされるとコンパイルエラーになることがあります。<br>
例えば、`Test.h`というファイルに、以下のように`Test`という構造体が定義されているとします。

```cpp
struct Test { int n; };
```

この`Test.h`を次のように2回インクルードすると、コンパイルエラーになります。

```cpp
#include "Test.h"
#include "Test.h" // ここでコンパイルエラー

int main() {
}
```

なぜなら、これは次のように書いたのと同じことだからです。

```cpp
struct Test { int n; };
struct Test { int n; }; // NG. 同じ名前の構造体を定義している

int main() {
}
```

このような分かりやすい例を見ると、「2回以上インクルードしないように注意すればいいのでは？」と思うかもしれません。<br>
ですが、多くの場合、それは無理なのです。

`EasyVec.h`を考えてみましょう。<br>
`EasyVec`クラスは基本的な配列クラスなので、別のさまざまなクラスの部品として使われることでしょう。

例えば、クラス`A`とクラス`B`がそれぞれ`EasyVec.h`をインクルードして使っているとします。<br>
そして、クラス`A`は`A.h`、クラス`B`は`B.h`に定義されているとしましょう。すると、次のようなプログラムを書きたくなるでしょう。

```cpp
#include "A.h"
#include "B.h"

int main() {
  A a;
  B b;
  // AとBを使ってなにか有用なことをする
}
```

一見すると何も問題はないように見えます。しかし、`A.h`も`B.h`も`EasyVec.h`をインクルードしています。<br>
そのため、「`EasyVec`クラスが2回定義されている」というコンパイルエラーになってしまうのです。

この問題を解決するには、「インクルード・ガード」と呼ばれる行を、ヘッダファイルに追加します。<br>
最初の`Test.h`にインクルードガードを追加すると、次のようになります。

```cpp
#ifndef TEST_H
#define TEST_H
// ↑ インクルードガード

struct Test { int n; };

// ↓ インクルードガード
#endif
```

追加した3つ行が「インクルード・ガード」です。
インクルードガードを追加した`Test.h`は、次のように2回インクルードしてもコンパイルエラーにはなりません。

```cpp
#include "Test.h"
#include "Test.h" // OK. エラーは起きない

int main() {
}
```

なぜなら、これは次のように書いたのと同じことだからです。

```cpp
#ifndef TEST_H // TEST_Hは定義されていないので、#endifまでの内容が有効になる
#define TEST_H // TEST_Hを定義

struct Test { int n; };

#endif // ここまで有効
#ifndef TEST_H // TEST_Hが定義されているので、#endifまでの内容は無視される
#define TEST_H

struct Test { int n; };

#endif // ここまで無視

int main() {
}
```

`#`で始まる行は「プリプロセッサ指令」といい、コンパイルの前に処理されます。<br>
インクルードガードでは、以下の3つのプリプロセッサ指令が使われます。

>```cpp
>#define マクロ名
>```
>
>`#define`(ディファイン)指令は、「マクロ名」という名前を定義します。

>```cpp
>#ifndef 式
>```
>
>`#ifndef`(イフ・エヌ・デフ)指令は、「式」が`0`以外の場合に、対応する`#endif`までのテキストを有効化します。

>```cpp
>#endif
>```
>
>`#endif`(エンド・イフ)指令は、対応する`#ifndef`の有効化・無効化範囲の終端です。

これらのプリプロセッサ指令の効果によって、インクルードガードは、2回目以降のインクルードの内容を無効化します。<br>
インクルードガードを使うことで、同じヘッダファイルを何度インクルードしても、コンパイルエラーにはなりません。

インクルードガードに使うマクロ名には、変数名と同じ文字制限はあるものの、それ以外は自由に決められます。<br>
ですが、同じ名前を使うことはできないため、重複を避けるために次のルールが使われることが多いです。

1. ヘッダファイル名をすべて大文字にする
2. `.`(ドット)を`_`(アンダーバー)に置き換える

このルールに従うと、`EasyVec.h`のインクルードガード用のマクロ名は`EASYVEC_H`となります。

それでは、`EasyVec.h`にインクルードガードを追加しましょう。`EasyVec.h`の先頭に、次のプログラムを追加してください。

**diff**

<pre><code><div><font color="#3a3">+#ifndef EASYVEC_H
+#define EASYVEC_H
+</font>
 /*
 * 配列クラス
 */
 template&lt;typename T&gt;
 class EasyVec {
</div></code></pre>

次に、`EasyVec.h`の末尾に、次のプログラムを追加してください。

**diff**

<pre><code><div>   T* first = nullptr; // 配列の先頭
   T* last = nullptr;  // 配列の終端
   T* cap = nullptr;   // 確保したメモリ領域の終端
   std::allocator&lt;T&gt; alloc; // メモリ管理用のオブジェクト
 };
<font color="#3a3">+
+#endif // EASYVEC_H</font>
</div></code></pre>

これで、`EasyVec.h`にインクルードガードが追加されました。

なお、インクルードガードの`#endif`には、コメントでマクロ名を書くことが多いです。<br>
関連するマクロ名を書いておくことで、インクルードガード以外の`#endif`がある場合に、範囲を間違えにくくなります。



In [ ]:
# @title 🔒️EasyVec.hの最終版(どうしてもバグが取れない場合に見てください)
%%writefile EasyVec.h
#ifndef EASYVEC_H
#define EASYVEC_H
#include <memory>

/*
* 配列クラス
*/
template<typename T>
class EasyVec {
public:
  // 型の定義
  using iterator = T*;
  using const_iterator = const T*;
  using size_type = size_t;
  using value_type = T;

  // コンストラクタ
  EasyVec() = default;
  explicit EasyVec(size_t n) {
    // メモリを確保し、配列の先頭を設定
     first = alloc.allocate(n);

     // 要素を初期化
     for (size_t i = 0; i < n; i++) {
       std::construct_at(first + i);
     }

     // 配列の終端を設定
     last = first + n;
  }

  // コピーコンストラクタ
  EasyVec(const EasyVec& other) {
    create(other);
  }

  // デストラクタ
  ~EasyVec() {
    destroy();
  }

  // コピー代入演算子
  EasyVec& operator=(const EasyVec& other) {
    // コピー元とコピー先が異なる場合だけコピーを実行
    if (this != &other) {

      if (size() != other.size()) {
        // サイズが異なる場合は配列を作り直す
        destroy();
        create(other);
      } else {
        // サイズが等しい場合はデータをコピーするだけ
        for (int i = 0; i < (int)size(); i++) {
          first[i] = other[i];
        }
      }
    } // if *this != other

    return *this; // このオブジェクトの参照を返す
  }

  // サイズの取得
  size_type size() const { return last - first; }

  // 添え字
  value_type& operator[](size_type i) { return first[i]; }
  const value_type& operator[](size_type i) const { return first[i]; }

  // イテレータ
  iterator begin() { return first; }
  const_iterator begin() const { return first; }
  iterator end() { return last; }
  const_iterator end() const { return last; }

  // 末尾に要素を追加する
  void push_back(const T& x) {
    // 未使用の部分が残っていなければ、配列のサイズを増やす
    if (last == cap) {
      grow();
    }

    // 配列の終端に要素を追加する
    std::construct_at(last, x);

    // 配列のサイズをひとつ増やす
    last++;
  }

  // 末尾の要素を削除する
  void pop_back() {
    // サイズが0でなければ要素をひとつ減らす
    if (size()) {
      std::destroy_at(last - 1);
      last--;
    }
  }

  // すべての要素を削除する
  void clear() {
    for (size_t i = 0; i < size(); i++) {
      std::destroy_at(first + i);
    }
    last = first;
  }

private:
  // 配列を作成する
  void create(const EasyVec& other) {
    // メモリを確保し、配列の先頭を設定
    const size_t n = other.size();
    first = alloc.allocate(n);

    // 要素をコピーコンストラクタで初期化
    for (size_t i = 0; i < n; i++) {
      std::construct_at(first + i, other[i]);
    }

    // 配列の終端を設定
    last = first + n;
    last = cap = first + n;
  }

  // 配列を破棄する
  void destroy() {
    // メモリを確保している場合のみ破棄と解放を実行
    if (first) {

      // 要素を破棄
      for (iterator i = first; i != last; i++) {
        std::destroy_at(i);
      }

      // メモリを解放
      alloc.deallocate(first, last - first);

      // ポインタを初期化
      first = last = nullptr;
      first = last = cap = nullptr;
    }
  }

  // 余分なメモリを確保する
  void grow() {
    // 確保するメモリのサイズを計算
    size_t n = 1; // メモリが確保されていない場合のサイズは1とする
    if (first) {
      // メモリが確保されている場合、元の2倍のメモリを確保する
      n = (cap - first) * 2;
    }

    // 新しいメモリを確保
    T* p = alloc.allocate(n);

    // 配列のすべてのデータを新しいメモリにコピー
    const size_t m = size();
    for (size_t i = 0; i < m; i++) {
      std::construct_at(p + i, first[i]);
    }

    // 元の配列を削除
    destroy();

    // 新しいメモリのアドレスとサイズをメンバ変数に反映
    first = p;
    last = p + m;
    cap = p + n;
  }

  T* first = nullptr; // 配列の先頭
  T* last = nullptr;  // 配列の終端
  T* cap = nullptr;   // 確保したメモリ領域の終端
  std::allocator<T> alloc; // メモリ管理用のオブジェクト
};

#endif // EASYVEC_H

----

## 2 練習問題

----

以下の手順にしたがって、各問題のプログラムを完成させなさい。

1. `%%writefile ...`の下の行からがプログラムです。問題文に従ってプログラムを修正、または追加してください。
2. プログラムを修正したら、セルの右側にある`▶`をクリックします。すると、ファイルが保存されます。
3. 「動作テスト」セルの`▶`をクリックすると、2で保存したファイルがコンパイル＆実行され、実行結果が表示されます。<br>
   このセルは、修正したプログラムの動作を確認するために使ってください。
4. 「実行」セルの`▶`をクリックすると、2で保存したファイルがコンパイル＆実行され、結果の成否が判定されます。
5. 判定に成功したら`AC`と表示されます。次の問題に進んでください。
6. 失敗したら`WA`と表示されます(その前にエラーメッセージが表示される場合もあります)。<br>
   これは、プログラムのどこかにエラーがあることを意味します。<br>
   「動作テスト」を使ってエラーを修正し、`AC`を目指してください。


### ❓️問題１ EasyVecの初期化と添え字

本テキストで作成した`EasyVec.h`を使って、配列にデータを読み込み、逆順で出力するプログラムを作成しなさい。<br>
最後に改行を出力すること。

>問題を解く前に、`EasyVec.h`セルを実行してファイルを保存すること。

**入力データ形式**

```txt
要素数N
整数A1 整数A2 ... 整数An
```

**入力例**

```txt
5
1 2 3 4 5
```

**出力例**

```txt
5 4 3 2 1
```


In [ ]:
%%writefile practice_01a.cpp
#include "EasyVec.h"
#include <iostream>
using namespace std;

int main() {
  // この下に、要素数と数値を読み込んで逆順で出力するプログラムを書く

}

In [ ]:
# @title 動作テスト
!g++ -std=c++20 -O2 -Wall -Wextra -o practice_01a practice_01a.cpp && echo "この下をクリックして、要素数と数値の配列を入力" && ./practice_01a

In [ ]:
# @title 実行
!diff -Z <(echo -e "5 4 3 2 1\n7") <(g++ -std=c++20 -O2 -Wall -Wextra -o practice_01a practice_01a.cpp && echo "5 1 2 3 4 5" | ./practice_01a && echo "1 7" | ./practice_01a) > /dev/null && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

In [ ]:
# @title 🔒解答例(どうしても問題を解けない場合に見てください)
%%writefile practice_01a.cpp
#include "EasyVec.h"
#include <iostream>
using namespace std;

int main() {
  // この下に、要素数と数値を読み込んで逆順で出力するプログラムを書く

  int n;
  cin >> n;

  EasyVec<int> v(n);
  for (int i = 0; i < n; i++) {
    cin >> v[i];
  }

  for (int i = n - 1; i >= 0; i--) {
    cout << v[i] << " ";
  }
  cout << endl;
}

### ❓️問題２ EasyVecのイテレータ

N個の整数 $ A_1 $ ～ $ A_N $ が入力されます。<br>
本テキストで作成した`EasyVec.h`を使って、配列にデータを読み込み、`sort`関数を使ってソートした結果を出力するプログラムを作成しなさい。最後に改行を出力すること。

>問題を解く前に、`EasyVec.h`セルを実行してファイルを保存すること。

**入力データ形式**

```txt
N
A1 A2 ... AN
```

**入力例**

```txt
5
3 1 4 1 5
```

**出力例**

```txt
1 1 3 4 5
```


In [ ]:
%%writefile practice_01b.cpp
#include "EasyVec.h"
#include <iostream>
#include <algorithm>
using namespace std;

int main() {
  // この下に、要素数と数値を読み込んでソートして出力するプログラムを書く

}

In [ ]:
# @title 動作テスト
!g++ -std=c++20 -O2 -Wall -Wextra -o practice_01b practice_01b.cpp && echo "この下をクリックして、要素数と数値の配列を入力" && ./practice_01b

In [ ]:
# @title 実行
!diff -Z <(echo -e "1 1 3 4 5\n6") <(g++ -std=c++20 -O2 -Wall -Wextra -o practice_01b practice_01b.cpp && echo "5 3 1 4 1 5" | ./practice_01b && echo "1 6" | ./practice_01b) > /dev/null && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

In [ ]:
# @title 🔒解答例(どうしても問題を解けない場合に見てください)
%%writefile practice_01b.cpp
#include "EasyVec.h"
#include <iostream>
#include <algorithm>
using namespace std;

int main() {
  // この下に、要素数と数値を読み込んでソートしてで出力するプログラムを書く

  int n;
  cin >> n;

  EasyVec<int> v(n);
  for (int i = 0; i < n; i++) {
    cin >> v[i];
  }

  sort(v.begin(), v.end());

  for (int i = 0; i < n; i++) {
    cout << v[i] << " ";
  }
  cout << endl;
}

### ❓️問題３ EasyVecのコピー

本テキストで作成した`EasyVec.h`について、コピーコンストラクタおよびコピー代入演算子の動作を確認しましょう。

N個の整数 $ A_1 $ ～ $ A_N $ が入力されます。<br>
次の仕様にしたがって、プログラムを完成させなさい。

**プログラムの仕様**

1. `int`型の変数`n`を宣言し、`cin`から`n`にデータ数を読み込む
2. `EasyVec<int>`型の変数`a`を宣言し、サイズ`n`で初期化する
3. for文を使って、`n`個の要素を`cin`から`a[i]`に読み込む
4. `EasyVec<int>`型の変数`b`を宣言し、変数`a`で初期化する
5. for文を使って、変数`b`の要素を2倍する
6. 変数`a`に変数`b`を代入する
7. for文を使って、変数`a`の内容を全て`cout`に出力する
8. `cout`に改行を出力する

>問題を解く前に、`EasyVec.h`セルを実行してファイルを保存すること。

**入力データ形式**

```txt
N
A1 A2 ... AN
```

**入力例**

```txt
3
1 2 3
```

**出力例**

```txt
2 4 6
```


In [ ]:
%%writefile practice_01c.cpp
#include "EasyVec.h"
#include <iostream>
using namespace std;

int main() {
  // この下に、1～8を行うプログラムを書く

}

In [ ]:
# @title 動作テスト
!g++ -std=c++20 -O2 -Wall -Wextra -o practice_01c practice_01c.cpp && echo "この下をクリックして、要素数と数値の配列を入力" && ./practice_01c

In [ ]:
# @title 実行
!diff -Z <(echo -e "2 4 6\n10 20 30 40 50 60 70 80\n0") <(g++ -std=c++20 -O2 -Wall -Wextra -o practice_01c practice_01c.cpp && echo "3 1 2 3" | ./practice_01c && echo "8 5 10 15 20 25 30 35 40" | ./practice_01c && echo "1 0" | ./practice_01c) > /dev/null && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

In [ ]:
# @title 🔒解答例(どうしても問題を解けない場合に見てください)
%%writefile practice_01c.cpp
#include "EasyVec.h"
#include <iostream>
using namespace std;

int main() {
  // この下に、1～8を行うプログラムを書く
  int n;
  cin >> n;

  EasyVec<int> a(n);
  for (int i = 0; i < n; i++) {
    cin >> a[i];
  }

  EasyVec<int> b = a;
  for (int i = 0; i < n; i++) {
    b[i] *= 2;
  }

  a = b;

  for (int i = 0; i < n; i++) {
    cout << a[i] << ' ';
  }
  cout << endl;
}

### ❓️問題４ EasyVecへの要素の追加と削除

配列を操作するN個の命令 $ C_1 $ ～ $ C_N $ が入力されます。命令には次の2種類があります。

* `a X`: 配列の末尾に整数Xを追加する
* `b`: 配列の末尾の要素を削除する

`EasyVec`クラスを使ってN個の命令をすべて実行し、すべての命令を実行した後の配列の内容を、出力するプログラムを作成しなさい。

**入力データ形式**

```txt
N
C1
C2
︙
CN
```

**入力例**

```txt
4
a 3
a 1
b
a 5
```

**出力例**

```txt
3 5
```


In [ ]:
%%writefile practice_02a.cpp
#include "EasyVec.h"
#include <iostream>
using namespace std;

int main() {
  // この下に、するプログラムを書く

}

In [ ]:
# @title 動作テスト
!g++ -std=c++20 -O2 -Wall -Wextra -o practice_02a practice_02a.cpp && echo "この下をクリックして、配列を操作する命令のリストを入力" && ./practice_02a

In [ ]:
# @title 実行
!diff -Z <(echo -e "3 5\n\n10 9 8 7 6 5 4 3 2 1") <(g++ -std=c++20 -O2 -Wall -Wextra -o practice_02a practice_02a.cpp && echo "4 a 3 a 1 b a 5" | ./practice_02a && echo "6 a 1 a 2 a 3 b b b" | ./practice_02a && echo "10 a 10 a 9 a 8 a 7 a 6 a 5 a 4 a 3 a 2 a 1" | ./practice_02a) > /dev/null && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

In [ ]:
# @title 🔒解答例(どうしても問題を解けない場合に見てください)
%%writefile practice_02a.cpp
#include "EasyVec.h"
#include <iostream>
using namespace std;

int main() {
  // この下に、するプログラムを書く
  EasyVec<int> v;

  int n;
  cin >> n;
  for (int i = 0; i < n; i++) {
    char a;
    cin >> a;
    if (a == 'a') {
      int b;
      cin >> b;
      v.push_back(b);
    } else {
      v.pop_back();
    }
  }

  for (auto i = v.begin(); i != v.end(); i++) {
    cout << *i << ' ';
  }
  cout << endl;
}

### ❓️問題５ EasyVecの全要素の消去

N個の文字 $ C_1 $ ～ $ C_N $ が入力されます。入力された順に、文字を配列に追加してください。<br>
ただし、入力された文字が小文字`c`の場合は文字を追加せず、配列の全要素を削除して空にしてください。

すべての文字を処理したあとの配列の内容を、出力するプログラムを作成しなさい。

**入力データ形式**

```txt
N
C1 C2 … CN
```

**入力例（１）**

```txt
5
1 2 3 c 4
```

**出力例（１）**

```txt
4
```

**入力例（２）**

```txt
10
a b c c d e c f g h
```

**出力例（２）**

```txt
f g h
```


In [ ]:
%%writefile practice_02b.cpp
#include "EasyVec.h"
#include <iostream>
using namespace std;

int main() {
  // この下に、N個の文字を処理した結果を出力するプログラムを書く

}

In [ ]:
# @title 動作テスト
!g++ -std=c++20 -O2 -Wall -Wextra -o practice_02b practice_02b.cpp && echo "この下をクリックして、文字数と文字のリストを入力" && ./practice_02b

In [ ]:
# @title 実行
!diff -Z <(echo -e "f g h\n\n4") <(g++ -std=c++20 -O2 -Wall -Wextra -o practice_02b practice_02b.cpp && echo "10 a b c c d e c f g h" | ./practice_02b && echo "1 c" | ./practice_02b && echo "5 1 2 3 c 4" | ./practice_02b) > /dev/null && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

In [ ]:
# @title 🔒解答例(どうしても問題を解けない場合に見てください)
%%writefile practice_02b.cpp
#include "EasyVec.h"
#include <iostream>
using namespace std;

int main() {
  // この下に、N個の文字を処理した結果を出力するプログラムを書く
  int n;
  cin >> n;

  EasyVec<char> v;
  for (int i = 0; i < n; i++) {
    char a;
    cin >> a;
    if (a == 'c') {
      v.clear();
    } else {
      v.push_back(a);
    }
  }

  for (auto i = v.begin(); i != v.end(); i++) {
    cout << *i << ' ';
  }
  cout << endl;
}

### ❓️問題６ 抜き取った番号

$ 1～N $ の数字の書かれたカードがそれぞれ3枚、合計で $ 3N $ 枚あります。<br>
コラボ君はこのカードから無作為に1枚を選んで抜き取り、残りの $ 3N - 1 $ 枚をあなたに返しました。

返されたカードに書かれた番号は $ A_1 $ ～ $ A_{3N-1} $ であらわされます(例えば $ N = 3 $ の場合は $ A_1 $ ～ $ A_8 $ )。<br>
コラボ君が抜き取ったカードに書かれている番号を出力するプログラムを作成しなさい。

**入力データ形式**

```txt
N
A1 A2 … A(3N-1)
```

**入力例（１）**

```txt
3
3 1 2 1 3 2 3 1
```

**出力例（１）**

```txt
2
```

**入力例（２）**

```txt
1
1 1
```

**出力例（２）**

```txt
1
```


In [ ]:
%%writefile practice_02c.cpp
#include "EasyVec.h"
#include <iostream>
using namespace std;

int main() {
  // この下に、するプログラムを書く

}

In [ ]:
# @title 動作テスト
!g++ -std=c++20 -O2 -Wall -Wextra -o practice_02c practice_02c.cpp && echo "この下をクリックして、数字の最大値と渡されるカードリストを入力" && ./practice_02c

In [ ]:
# @title 実行
!diff -Z <(echo -e "2\n1\n4") <(g++ -std=c++20 -O2 -Wall -Wextra -o practice_02c practice_02c.cpp && echo "3 3 1 2 1 3 2 3 1" | ./practice_02c && echo "1 1 1" | ./practice_02c && echo "9 3 9 8 3 8 2 9 1 6 8 5 1 4 1 2 7 7 4 7 2 5 3 9 6 6 5" | ./practice_02c) > /dev/null && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

In [ ]:
# @title 🔒解答例(どうしても問題を解けない場合に見てください)
%%writefile practice_02c.cpp
#include "EasyVec.h"
#include <iostream>
using namespace std;

int main() {
  // この下に、するプログラムを書く
  int n;
  cin >> n;

  EasyVec<int> v(n);
  for (int i = 0; i < n * 3 - 1; i++) {
    int a;
    cin >> a;
    v[a - 1]++;
  }

  for (int i = 0; i < n; i++) {
    if (v[i] == 2) {
      cout << i + 1 << endl;
      break;
    }
  }
}

### ❓️問題７ 試験の順位

$ N $ 人が試験を受けました。試験の得点のリストは $ A_1 ～ A_N $ であり、 $ i $ 人目の得点は $ N_i $ です。

受験者の順位は次のルールで決められます。

1. N人全員の順位を「未確定」とする
2. 順位`r`を`1`に設定する
3. すべての人の順位が「確定」 するまで、以下の手順を繰り返す
    1. 順位が「未確定」の人の中で最高得点を選ぶ
    2. 最高得点と等しい人の順位を`r`に「確定」する
    3. 手順4で順位が「確定」した人数を`r`に足す

確定した順位を出力するプログラムを作成しなさい。

**入力データ形式**

```txt
N
A1 A2 … AN
```

**入力例（１）**

```txt
4
6 3 7 6
```

**出力例（１）**

```txt
2 3 1 2
```

**入力例（２）**

```txt
2
10 10
```

**出力例（２）**

```txt
1 1
```


In [ ]:
%%writefile practice_02c.cpp
#include "EasyVec.h"
#include <iostream>
#include <algorithm>
using namespace std;

int main() {
  // この下に、順位を出力するプログラムを書く

}

In [ ]:
# @title 動作テスト
!g++ -std=c++20 -O2 -Wall -Wextra -o practice_02c practice_02c.cpp && echo "この下をクリックして、人数と得点リストを入力" && ./practice_02c

In [ ]:
# @title 実行
!diff -Z <(echo -e "2 3 1 5 3\n1\n13 13 19 13 8 1 1 13 19 10 6 6 3 3 10 3 10 13 8 13") <(g++ -std=c++20 -O2 -Wall -Wextra -o practice_02c practice_02c.cpp && echo "5 10 8 11 5 8" | ./practice_02c && echo "1 100" | ./practice_02c && echo "20 2 2 0 2 6 10 10 2 0 3 8 8 9 9 3 9 3 2 6 2" | ./practice_02c) > /dev/null && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

In [ ]:
# @title 🔒解答例(どうしても問題を解けない場合に見てください)
%%writefile practice_02c.cpp
#include "EasyVec.h"
#include <iostream>
#include <algorithm>
using namespace std;

int main() {
  // この下に、順位を出力するプログラムを書く
  int n;
  cin >> n;

  EasyVec<int> v(n);
  for (int i = 0; i < n; i++) {
    cin >> v[i];
  }

  EasyVec<int> rank(n);
  for (int r = 1; r <= n; ) {

    int max_score = 0;
    for (int i = 0; i < n; i++) {
      if (rank[i] == 0 && v[i] > max_score) {
        max_score = v[i];
      }
    }

    int k = 0;
    for (int i = 0; i < n; i++) {
      if (v[i] == max_score) {
        rank[i] = r;
        k++;
      }
    }
    r += k;
  }

  for (int i = 0; i < n; i++) {
    cout << rank[i] << ' ';
  }
  cout << endl;
}